# Build A Rocket And Launch It

Proceduarlly build and sim a flight. This is my attempt to use the [open aerospace rocket documentation tool](https://open-aerospace.github.io/openrocketdoc/) to describe a rocket and generate JSBSim configuration to simulate its flight.

## Design Engine

Pick an engine design.

In [1]:
thrust     = 1255.0  # N
burn_time  =   12.0  # s
isp        =  214.0  # s

Build the engine:

In [2]:
from openrocketdoc import document
from openrocketdoc import writers

engine = document.Engine('Engine')

engine.Isp = isp
engine.thrust_avg = thrust
engine.t_burn = burn_time

# The Open Rocket Document can compute useful values based on what we defined above.
print("Our engine has %0.1f kg of propellent\n" % engine.m_prop)
print("Generated JSBSim thrust curve:\n\n```")
print(writers.JSBSimEngine.dump(engine), "```")

Our engine has 7.2 kg of propellent

Generated JSBSim thrust curve:

```
<?xml version="1.0" ?>
<rocket_engine name="Engine">
  <isp>214.0</isp>
  <builduptime>0.1</builduptime>
  <thrust_table name="propulsion/thrust_prop_remain" type="internal">
    <tableData>
 0.000 282.135
 5.274 282.135
 10.547 282.135
    </tableData>
  </thrust_table>
</rocket_engine>
 ```


## Build Rocket

Now we know how much propellent, guess the density and come up with some parametric rocket design.

In [3]:
from math import pi
prop_density = 1750      # kg/m3  Roughtly HTPB composit solid density[1]
LD           =   10      # Length to width ratio
# [1] http://www.lr.tudelft.nl/en/organisation/departments/space-engineering/space-systems-engineering/expertise-areas/space-propulsion/design-of-elements/rocket-propellants/solids/

# volume of propellent needed
prop_volume = engine.m_prop/prop_density

# Solve for the radius/length of the fuel grain (assume solid, end burning)
engine.diamter = 2*(prop_volume/ (2*LD*pi))**(1/3.0)
engine.length = engine.diamter * LD

# Add a nose
nosecone = document.Nosecone(
    document.Noseshape.TANGENT_OGIVE,  # Shape
    1.0, # shape_parameter
    1.5, # mass
    engine.diameter * 6, # length, 6:1 L/D
    diameter=engine.diameter,
    material_name="Aluminium"
)

# Body section the size of the engine
body = document.Bodytube(
    "Body",  # Name
    1.5,     # mass
    engine.length, 
    diameter=engine.diameter,
    material_name="Aluminium"
)

# Rocket:
rocket = document.Rocket("Rocket")
stage0 = document.Stage("Sustainer")
stage0.components = [nosecone, body]
rocket.stages = [stage0]

# Print:
print("Generated JSBSim 'Aircraft':\n\n```")
print(writers.JSBSimAircraft.dump(rocket))
print("```")

Generated JSBSim 'Aircraft':

```
<?xml version="1.0" ?>
<fdm_config name="Rocket" release="ALPHA" version="2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://jsbsim.sourceforge.net/JSBSim.xsd">
  <fileheader/>
  <metrics>
    <wingarea unit="M2">0.0</wingarea>
    <wingspan unit="M">0.0</wingspan>
    <chord unit="M">0.0</chord>
    <htailarea unit="M2">0.0</htailarea>
    <htailarm unit="M">0.0</htailarm>
    <vtailarea unit="M2">0.0</vtailarea>
    <vtailarm unit="M">0.0</vtailarm>
    <location name="AERORP" unit="M">
      <x>0.0</x>
      <y>0.0</y>
      <z>0.0</z>
    </location>
  </metrics>
  <mass_balance>
    <pointmass name="Body">
      <form shape="tube">
        <radius unit="M">0.0000</radius>
        <length unit="M">0.8052</length>
      </form>
      <weight unit="KG">1.5000</weight>
      <location unit="M">
        <x>0.4026</x>
        <y>0.0</y>
        <z>0.0</z>
      </location>
    </pointmass>
  </mass_balance>
